# Numerical Analysis Math 4610

## Assignment 5

-----

#### Supporting Functions

In [1]:
#imports and settings
import numpy as np
np.set_printoptions(suppress=True, linewidth=1000, precision=2)

In [2]:
#returns the the absolute error of the approximation
def absoluteError(f1, f2, x_0):
    return abs(f1(x_0) - f2(x_0))

In [10]:
#Hermite Cubic Spline
def HermiteInterpolation(x,y,dy):
    """
    Hermite Interpolation

    Args:
        x (np.array): ordinal values
        y (np.array): function values
        dy (np.array): derivative values

    Returns:
        np.array: coefficients of the Hermite polynomial
    """
    n = len(x)
    X = np.repeat(x,2)
    Y = np.repeat(y,2)
    q = np.zeros((2*n,2*n+1))

    for i in range(0, 2 * n, 2):
        idx = i//2
        q[i][0] = x[idx]
        q[i+1][0] = x[idx]
        q[i][1] = y[idx]
        q[i+1][1] = y[idx]

    for i in range(2, 2*n+1):
        for j in range(1+(i-2), 2*n):
            if i == 2 and j%2 == 1:
                q[j][i] = dy[j//2]
            else:
                q[j][i] = (q[j][i-1] - q[j-1][i-1]) / (q[j][0] - q[(j-1)-(i-2)][0])
    return q

In [11]:
#Cubic Spline Interpolation
def CubicSpline(x,a):
    """_summary_

    Args:
        x (np.array): ordinal values
        a (np.array): function values

    Returns:
        np.array: coefficients of the cubic spline polynomial
    """
    n = len(x)
    h = np.diff(x)
    
    alpha = np.zeros(n)
    
    for i in range(1, n-1):
        alpha[i] = (3/h[i])*(a[i+1]-a[i]) - (3/h[i-1])*(a[i]-a[i-1])
    
    l = np.zeros(n)
    u = np.zeros(n)
    z = np.zeros(n)
    c = np.zeros(n)
    b = np.zeros(n)
    d = np.zeros(n)
    l[0] = 1

    for i in range(1, n-1):
        l[i] = 2*(x[i+1]-x[i-1]) - h[i-1]*u[i-1]
        u[i] = h[i]/l[i]
        z[i] = (alpha[i] - h[i-1]*z[i-1])/l[i]
    l[n-1] = 1
    
    
    for j in range(n-2, -1, -1):
        c[j] = z[j] - u[j]*c[j+1]
        b[j] = (a[j+1]-a[j])/h[j] - h[j]*(c[j+1]+2*c[j])/3
        d[j] = (c[j+1]-c[j])/(3*h[j])
        
    print(c, d, b)
    return np.array([a, b, c, d])



### Section 3.4

1. Use Theorem 3.9 or Algorithm 3.3 to construct an approximating polynomial for the following data.

$(a)$


| $x$ | $f(x)$ | $f’(x)$ |
|---|------|-------|
| $0$ | $1.00000$ | $2.00000$ |
| $0.5$ | $2.71828$ | $5.43656$ |

In [13]:
x = np.array([0,0.5])
y = np.array([1,2.71828])
dy = np.array([2,5.43656])
HermiteInterpolation(x,y,dy)

array([[0.  , 1.  , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 2.  , 0.  , 0.  ],
       [0.5 , 2.72, 3.44, 2.87, 0.  ],
       [0.5 , 2.72, 5.44, 4.  , 2.25]])

$H_3(x) = 1 + 2(x-0) + 2.8731(x-0)^2 + 2.2538(x-0)^2(x-.5)$

$(b)$

| $x$ | $f(x)$ | $f’(x)$ |
|---|------|-------|
| $0.1$ | $-0.29004996$ | $ -2.8019975$ |
| $0.2$ | $-0.56079734$ | $-2.6159201$ |
| $0.3$ | $-0.81401972$ | $-2.9734038$ |

In [14]:
x = np.array([0.1, 0.2, 0.3])
y = np.array([-0.29004996, -0.56079734, -0.81401972])
dy = np.array([-2.8019975,-2.6159201,-2.9734038])
HermiteInterpolation(x,y,dy)

array([[    0.1 ,    -0.29,     0.  ,     0.  ,     0.  ,     0.  ,     0.  ],
       [    0.1 ,    -0.29,    -2.8 ,     0.  ,     0.  ,     0.  ,     0.  ],
       [    0.2 ,    -0.56,    -2.71,     0.95,     0.  ,     0.  ,     0.  ],
       [    0.2 ,    -0.56,    -2.62,     0.92,    -0.3 ,     0.  ,     0.  ],
       [    0.3 ,    -0.81,    -2.53,     0.84,    -0.39,    -0.48,     0.  ],
       [    0.3 ,    -0.81,    -2.97,    -4.41,   -52.49,  -260.47, -1299.97]])

$H_5(x) = -.29 - 2.802(x-.1) + 0.9452(x-.1)^2 - 0.297(x-.1)^2(x-.2) - 0.4794(x-.1)^2(x-.2)^2 - 1299.9722(x-.1)^2(x-.2)^2(x-.3)$



2.The data in previous exercise (above) were generated using the following functions. Use the polynomials constructed in previous problem for the given value of $x$ to approximate $f(x)$, and calculate the absolute error. (a)$f(x) =e^{2x}$; approximate $f(0.43)$. (b)$f(x) =x^2 cosx− 3 x$; approximate $f(0.18).$

$\\$

$a)$ $f(x) = e^{2x}$; $f(0.43) = 2.3631606937057947$

The absolute error for $H_3$ is:

$|f(0.43) - H_3(0.43)| = 0.0010954371057949608$

In [15]:
def f1(x_0):
    return np.exp(2*x_0)
def f2(x_0):
    return 1 + 2*(x_0-0) + 2.8731*(x_0-0)**2 + 2.2538*(x_0-0)**2*(x_0-0.5)
absoluteError(f1, f2, .43)

0.0010954371057949608

$\\$

$b)$ $f(x) = x^2cosx -3x$; $f(0.18) = -0.508123464354$

The absolute error for $H_5$ is:

$|f(1.8) - H_5(1.8)| = 0.0004488845495050908$

In [16]:
def f1(x):
    return x**2 * np.cos(x)-3*x
def f2(x):
    y =  -.29 + -2.802*(x-.1)
    y = y + (0.9452)*(x-.1)**2 
    y = y + (-0.297)*(x-.1)**2*(x-.2)
    y = y + (-0.4794)*(x-.1)**2*(x-.2)**2
    y = y - 1299.9722*(x-.1)**2*(x-.2)**2*(x-.3)
    return y
absoluteError(f1, f2, .18)



0.0004488845495050908

$\\$

## Section 3.5

1.Determine the clamped cubic splines that interpolates the data
$f(0) = 0$, $f(1) = 1$, $f(2) = 2$ and satisfies $s′(0) =s′(2) = 1$.

Because $f$ is defined at $a = 0 < 1 < 2 = b$ and is differentiable at $a$ and $b$, then $f$ has a unique clamped spline interpolated at $0,1,2$ that satisfies $s′(0) = f′(0)$  and $s'(2) = f'(2)$.

points:
- $(0,0), (1,1), (2,2)$
- $s'(0) = s'(2) = 1$

$S_i = a_i + b_i(x-x_i) + c_i(x-x_i)^2 + d_i(x-x_i)^3, x_i \le x \le x_{i+1}$

Spline $[0,1]$: 
$S_0 = a_0 + b_0(x-0) + c_0(x-0)^2 + d_0(x-0)^3$

Spline $[1,2]$
$S_1 = a_1 + b_1(x-1) + c_1(x-1)^2 + d_1(x-1)^3$

-----

$h_n = x_{n+1} - x_n$

$h_0 = 1 - 0 = 1$

$h_1 = 2 - 1 = 1$

-----

$A = \begin{bmatrix} 2h_0 & h_0 & 0 \\ h_0 & 2(h_0+h_1) & h_1 \\ 0 & h_1 & 2h_{n-1} \end{bmatrix}$

$\implies A = \begin{bmatrix} 2 & 1 & 0 \\ 1 & 4 & 1 \\ 0 & 1 & 2 \end{bmatrix}$

 $u = \begin{bmatrix} \frac{3}{h_0}(a_1-a_0) - 3f'(a)\\ \frac{3}{h_1}(a_2-a_1) - 3f'(a) \\ 3f'(b)-\frac{3}{h_1}(a_2-a_1) \end{bmatrix}$

$\implies \begin{bmatrix} 2 & 1 & 0 \\ 1 & 4 & 1 \\ 0 & 1 & 2 \end{bmatrix} * \begin{bmatrix} c_0\\ c_1 \\ c_2\end{bmatrix} = \begin{bmatrix} \frac{3}{1}(1) - 3\\ \frac{3}{1}(1) - 3 \\ 3-\frac{3}{1}(1) \end{bmatrix}$


$\implies \begin{bmatrix} 2 & 1 & 0 \\ 1 & 4 & 1 \\ 0 & 1 & 2 \end{bmatrix} \begin{bmatrix} 0\\ 0 \\ 0 \end{bmatrix}$

$\implies \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} 0\\ 0 \\ 0 \end{bmatrix}$

$\implies c_0 = c_1 = c_2=0$

-----

$b_j = \frac{1}{h_j}(a_{j+1} - a_j) - \frac{h_j}{3}(2c_j + c_{j+1})$

$\implies b_0 = \frac{1}{1}(1-0) - \frac{1}{3}(2*0 + 0),$

$\implies b_1 = \frac{1}{1}(2-1) - \frac{1}{3}(2*0 + 0)$

$\implies b_0 = b_1 = 1, b_2 = 0$

-----

$d_j = \frac{c_{j+1} - c_j}{3h_j}$

$\implies d_0 = \frac{0 - 0}{3*1} = 0$

$\implies d_1 = \frac{0 - 0}{3*1} = 0$

$\implies d_0 = d_1 = d_2 = 0$

-----

$a_0 = 0; a_1 = 1; a_2 = 2$

$b_0 = 1; b_1 = 1; b_2 = 0$

$c_0 = 0; c_1 = 0; c_2 = 0$

$d_0 = 0; d_1 = 0; d_2 = 0$

-----

$ S(x) = $


$ S_0 = 0 + 1(x-0) + 0(x-0)^2 + 0(x-0)^3; 0 \le x < 1 $

$ S_1 = 1 + 1(x-1) + 0(x-1)^2 + 0(x-1)^3; 1 \le x < 2 $

$ S(x) = $

$ S_0 = x; 0 \le x < 1 $

$ S_1 = x; 1 \le x < 2 $


2. Construct the natural cubic spline for the following data.

$(a)$

|$x$| $f(x)$|
|---|------|
|$0$| $1.00000$|
|$0.5$| $2.71828$|

In [84]:
x = np.array([0.0, 0.5])
y = np.array([1.0, 2.71828])
CubicSpline(x,y)

array([[1.    , 2.7183],
       [3.4366, 0.    ],
       [0.    , 0.    ],
       [0.    , 0.    ]])

$ S(x) = $
$ 1 + 3.4366(x) + 0(x)^2 + 0(x)^3;$ $0 \le x < 0.5 $

$ 2.7183 + 0(x) + 0(x)^2 + 0(x)^3;$ $x \ge .5 $

$(b)$

| $x$ | $f(x)$ | $f’(x)$|
|---|------|-------|
| $0.1$ | $-0.29004996$ | $-2.7513$ |
| $0.2$ | $-0.56079734$ | $-2.6198$ |
| $0.3$ | $-0.81401972$ | $-2.4884$ |


In [85]:
x = np.array([0.1, 0.2, 0.3])
y = np.array([-0.29004996, -0.56079734, -0.81401972])
CubicSpline(x,y)

array([[-0.29  , -0.5608, -0.814 ],
       [-2.7513, -2.6198,  0.    ],
       [ 0.    ,  1.3144,  0.    ],
       [ 4.3812, -4.3812,  0.    ]])

$ S(x) = $

$ -0.29 - 2.7513(x - .1) + 4.3812(x-.1)^3; $ $ 0.1 \le x < 0.2 $

$ -0.5608 - 2.6198(x-.2) + 1.3144(x - .2)^2 - 4.3812(x-.2)^3 $  $ 0.2 \le x < 0.3 $




$\\$